### 1. pandas 및 pickle 불러오기

In [90]:
import pandas as pd
import numpy as np
import os
import sklearn.metrics
import pickle as pickle



num_to_label pickle 에서 불러오려면 아래 셀 uncomment 후 경로 지정

num_to_label dict 직접 만들기

In [2]:
num_to_label = {0: 'no_relation', 1: 'org:top_members/employees', 2: 'org:members', 3: 'org:product', 4: 'per:title', 5: 'org:alternate_names', 6: 'per:employee_of', 7: 'org:place_of_headquarters', 8: 'per:product', 9: 'org:number_of_employees/members', 10: 'per:children', 11: 'per:place_of_residence', 12: 'per:alternate_names', 13: 'per:other_family', 14: 'per:colleagues', 15: 'per:origin', 16: 'per:siblings', 17: 'per:spouse', 18: 'org:founded', 19: 'org:political/religious_affiliation', 20: 'org:member_of', 21: 'per:parents', 22: 'org:dissolved', 23: 'per:schools_attended', 24: 'per:date_of_death', 25: 'per:date_of_birth', 26: 'per:place_of_birth', 27: 'per:place_of_death', 28: 'org:founded_by', 29: 'per:religion'}

In [91]:
def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']
    no_relation_label_idx = label_list.index("no_relation")
    preds = label_to_num(preds.values)
    labels = label_to_num(labels.values)

    label_indices = list(range(len(label_list)))
    label_indices.remove(no_relation_label_idx)
    return sklearn.metrics.f1_score(labels, preds, average="micro", labels=label_indices) * 100.0


def label_to_num(label):
  num_label = []
  with open('../code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  
  return num_label

In [94]:
def mixmaster(df1, df2, weight1, weight2):
    result = []
    df1= np.array(list(map(float, df1[1:-1].split(', ')))) * weight1
    df2= np.array(list(map(float, df2[1:-1].split(', ')))) * weight2
    result =np.mean([df1, df2], axis=0)
    result = result/result.sum()
    result = '[' + ', '.join(map(str, result)) + ']'
    return result

In [95]:
def change_label(x):
    x= np.array(list(map(float, x[1:-1].split(', '))))
    return num_to_label[np.argmax(x)]

In [105]:
def ensemble(folder_name, file_name1, file_name2, weight1 = 1, weight2 = 1):
    ens1 = pd.read_csv(f'{folder_name}/{file_name1}')
    ens2 = pd.read_csv(f'{folder_name}/{file_name2}')
    result = ens1.copy()
    
    result.probs=result.id.apply(lambda x: mixmaster(ens1.probs.iloc[x],ens2.probs.iloc[x],weight1,weight2))
    
    result.pred_label= result.probs.apply(lambda x: change_label(x))
    
    return result
    
    

In [106]:
folder_name = './ensemble'
file_list = os.listdir(folder_name)
count_file = len(file_list)

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

f1_scores={}

for file_1 in range(count_file):
    for file_2 in range(file_1+1,count_file):
        
        print(file_1 == file_2,file_1 , file_2)
        file_name = f'{file_list[file_1][6:-4]}_{file_list[file_2][6:-4]}'
        ens_result = ensemble(folder_name,file_list[file_1],file_list[file_2])
        ens_result.to_csv(f'./ensemble_result/{file_name}.csv')
        ens_result= pd.concat([ens_result,correct],axis=1)
        f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))     



False 0 1
False 0 2
False 1 2


In [111]:
folder_name = './ensemble'
file_list = os.listdir(folder_name)
count_file = len(file_list)
os.mkdir('./ensemble_result')

correct = pd.read_csv('../data/dataset/test/dev_final.csv')

f1_scores={}
file_name1= 'train_14.csv'
file_name2 = 'train_14_roberta-large.csv'

for weight1 in range(1,6):

    print(weight1)

    file_name = f'{file_name1[6:-4]}_{weight1}_{file_name2[6:-4]}_5'
    ens_result = ensemble(folder_name,file_name1,file_name2,weight1,5)
    ens_result.to_csv(f'./ensemble_result/{file_name}.csv')
    ens_result= pd.concat([ens_result,correct],axis=1)
    f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

for weight2 in range(1,6):

    print(weight2)

    file_name = f'{file_name1[6:-4]}_5_{file_name2[6:-4]}_{weight2}'
    ens_result = ensemble(folder_name,file_name1,file_name2,5,weight2)
    ens_result.to_csv(f'./ensemble_result/{file_name}.csv')
    ens_result= pd.concat([ens_result,correct],axis=1)
    f1_scores[file_name]=klue_re_micro_f1(ens_result.pred_label,ens_result.label)

f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))
f1_scores


1
2
3
4
5
1
2
3
4
5


In [108]:
f1_scores=dict(sorted(f1_scores.items(), key=lambda x:-x[1]))

In [112]:
f1_scores

{'14_5_14_roberta-large_5': 70.74960127591706,
 '14_4_14_roberta-large_5': 70.69186418962204,
 '14_3_14_roberta-large_5': 70.46947604550552,
 '14_5_14_roberta-large_4': 70.40686586141132,
 '14_2_14_roberta-large_5': 70.0095816033216,
 '14_1_14_roberta-large_5': 69.69407265774379,
 '14_5_14_roberta-large_3': 69.48000632211159,
 '14_5_14_roberta-large_2': 68.75491893593578,
 '14_5_14_roberta-large_1': 68.1023720349563}